# Logging Practices and Water Quality

## Setup

In [ ]:
# 3rd party library imports
from IPython.display import Markdown as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

sns.set()

df = pd.read_csv('case1501.csv')

## Nitrates ($NO_3$ - $N$) in runoff from patch-cut and undisturbed watersheds, for 5 years after logging

In [ ]:
fig, axes = plt.subplots(nrows=2)
ax0 = sns.lineplot(data=df, x='Week', y='Patch', ax=axes[0])
ax1 = sns.lineplot(data=df, x='Week', y='NoCut', ax=axes[1])
ax0.set_ylim(0, 5)
ax1.set_ylim(0, 5)
fig.suptitle('Nitrate Levels')

## Estimating the 1st serial correlation coefficient

In [ ]:
mp = df['Patch'].mean()
npatch = len(df['Patch'])
dof = npatch - 1

res = df['Patch'] - mp
reslag = df['Patch'].shift(1) - mp
c0_patch = df['Patch'].std() ** 2
c1_patch = (res * reslag)[1:].sum() / dof

mn = df['NoCut'].mean()
nnocut = len(df['NoCut'])

res = df['NoCut'] - mn
reslag = df['NoCut'].shift(1) - mn
c0_nocut = df['NoCut'].std() ** 2
c1_nocut = (res * reslag)[1:].sum() / dof

c0 = (dof * c0_patch + dof * c0_nocut) / (dof + dof)
c1 = (dof * c1_patch + dof * c1_nocut) / (dof + dof)
r1 = c1 / c0
r1

In [ ]:
sp = np.sqrt(((npatch - 1) * df['Patch'].std() ** 2 + (nnocut - 1) * df['NoCut'].std() ** 2) / (npatch + nnocut - 2))
se_diff = np.sqrt((1 + r1) / (1 - r1)) * sp * np.sqrt(1 / npatch + 1 / nnocut)
se_diff

## Is serial correlation even present?

In [ ]:
resid = df['NoCut'] - df['NoCut'].mean()
num_runs = (np.abs(np.sign(resid).diff()) > 0).sum() + 1
m = (resid < 0).sum()
p = (resid > 0).sum()
mu = (2 * m * p) / ( m + p) + 1
sigma = np.sqrt((2 * m * p) * (2 * m * p - m - p) / ((m + p) ** 2 * (m + p + 1)))
z = (num_runs - mu + 0.5) / sigma
p = (1 - norm.cdf(np.abs(z))) * 2
p

There is strong evidence of a serial effect ($Z = -6.398$, $p < 0.0001$)